# Interactive plots of LaueNN results for one/two phase material
## plotting of laueNN results without the LaueNN library

In [1]:
## install libraries in the jupyter notebook for the current kernel using the line below (remove the #)
#!pip install ipympl

In [4]:
## Lets load the library and results file
#%matplotlib inline
%matplotlib widget
import _pickle as cPickle
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
import numpy as np, re
from ipywidgets import interact, widgets, interactive
from IPython.display import display

prediction_GUI = False
### USER INPUT
### Please enter the path of the results.npz file (which is extracted from the LauetoolsNN results directory)
result_directory = r"E:\Ti_NGey\Ti_alpha_Ti_beta\results_Ti_alpha_2022-07-09_00-29-44"
results_obj = np.load(result_directory + "//results.npz", allow_pickle=True)
print("Results loaded successfully")


best_match = results_obj["arr_0"] # contains all the details regarding the indexation
mat_global = results_obj["arr_1"] # material id of each pixel (to identify if it is Si or Cu)
rotation_matrix = results_obj["arr_2"] # 3x3 rotation matrix for each pixel (in Lauetools frame)
strain_matrix = results_obj["arr_3"] # 3x3 strain matrix of each UB matrix for each pixel (crystal reference frame)
strain_matrixs = results_obj["arr_4"] # 3x3 strain matrix of each UB matrix for each pixel (sample reference frame)
col = results_obj["arr_5"] # Color in sample Z direction confroming the IPF color (be careful)
colx = results_obj["arr_6"] # Color in sample X direction confroming the IPF color (be careful)
coly = results_obj["arr_7"] # Color in sample Y direction confroming the IPF color (be careful)
match_rate = results_obj["arr_8"] # Matching rate of each indexation
files_treated = results_obj["arr_9"] # File names of each pixel
lim_x = results_obj["arr_10"] # Limit X of raster scan
lim_y = results_obj["arr_11"] # Limit Y of raster scan
spots_len = results_obj["arr_12"] # Number of spots per pattern indexed
iR_pix = results_obj["arr_13"] # initial pixel residues before strain refinement
fR_pix = results_obj["arr_14"] # final residue pixel after strain refinement
ub_matricies = len(rotation_matrix) # Total UB matricies
if prediction_GUI:
    material_ = str(results_obj["arr_15"]) # Material name for mat id 1
    material1_ = str(results_obj["arr_16"]) # Material name for mat id 2
    material = [material_, material1_]
else:
    material = str(results_obj["arr_15"])

filenames = list(np.unique(files_treated))
filenames.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
print("number of UB matricies in the results", ub_matricies)
print("material present", material)

## compute additional variables such as lattice parameters and misorientation maps beforehand here if you wish to plot them
## misorientation (see for example misorientation_relative_plots.py script --> be careful with this script, needs verification)
## lattice parameters (see for example unit_cell_relative_plots.py script)


Results loaded successfully
number of UB matricies in the results 4
material present ['Ti_alpha', 'Ti_beta']


## Histrogram plots

In [6]:
def f1(plot_ = "select a reference frame", match_tol0=0, pixel_residues0=1, NbofSpots0=0, bins0=50, \
                match_tol1=0, pixel_residues1=1, NbofSpots1=0, bins1=50):
    
    if plot_ == "select a reference frame":
        print("please select a reference frame from the dropdown list")
        return
    
    for matid in range(len(material)):
        count = 0
        
        if matid == 0:
            match_tol = match_tol0
            pixel_residues = pixel_residues0
            NbofSpots = NbofSpots0
            bins = bins0
        elif matid == 1:
            match_tol = match_tol1
            pixel_residues = pixel_residues1
            NbofSpots = NbofSpots1
            bins = bins1
        
        for index in range(ub_matricies):
            ### index for nans
            nan_index1 = np.where(match_rate[index][0] < match_tol)[0]
            mat_id_index = np.where(mat_global[index][0] != matid+1)[0]
            nan_index2 = np.where(fR_pix[index][0].flatten() > pixel_residues)[0]
            nan_index3 = np.where(spots_len[index][0] < NbofSpots)[0]
            nan_index = np.hstack((mat_id_index,nan_index1,nan_index2,nan_index3))
            nan_index = np.unique(nan_index)

            if count == 0:
                spots_len_plot = np.copy(spots_len[index][0])
                mr_plot = np.copy(match_rate[index][0])
                iR_pix_plot = np.copy(iR_pix[index][0])
                fR_pix_plot = np.copy(fR_pix[index][0])
                strain_matrix_plot = np.copy(strain_matrix[index][0])
                e11c = strain_matrix_plot[:,0,0]#.reshape((lim_x, lim_y))
                e22c = strain_matrix_plot[:,1,1]#.reshape((lim_x, lim_y))
                e33c = strain_matrix_plot[:,2,2]#.reshape((lim_x, lim_y))
                e12c = strain_matrix_plot[:,0,1]#.reshape((lim_x, lim_y))
                e13c = strain_matrix_plot[:,0,2]#.reshape((lim_x, lim_y))
                e23c = strain_matrix_plot[:,1,2]#.reshape((lim_x, lim_y))
                strain_matrixs_plot = np.copy(strain_matrixs[index][0])
                e11s = strain_matrixs_plot[:,0,0]#.reshape((lim_x, lim_y))
                e22s = strain_matrixs_plot[:,1,1]#.reshape((lim_x, lim_y))
                e33s = strain_matrixs_plot[:,2,2]#.reshape((lim_x, lim_y))
                e12s = strain_matrixs_plot[:,0,1]#.reshape((lim_x, lim_y))
                e13s = strain_matrixs_plot[:,0,2]#.reshape((lim_x, lim_y))
                e23s = strain_matrixs_plot[:,1,2]#.reshape((lim_x, lim_y))
                spots_len_plot[nan_index] = np.nan 
                mr_plot[nan_index] = np.nan 
                iR_pix_plot[nan_index] = np.nan 
                fR_pix_plot[nan_index] = np.nan 
                e11c[nan_index] = np.nan 
                e22c[nan_index] = np.nan 
                e33c[nan_index] = np.nan 
                e12c[nan_index] = np.nan 
                e13c[nan_index] = np.nan 
                e23c[nan_index] = np.nan 
                e11s[nan_index] = np.nan 
                e22s[nan_index] = np.nan 
                e33s[nan_index] = np.nan 
                e12s[nan_index] = np.nan 
                e13s[nan_index] = np.nan 
                e23s[nan_index] = np.nan 
                count = 1

            else:
                temp = np.copy(spots_len[index][0])
                temp[nan_index] = np.nan
                spots_len_plot = np.vstack((spots_len_plot,temp))

                temp = np.copy(match_rate[index][0])
                temp[nan_index] = np.nan
                mr_plot = np.vstack((mr_plot,temp))

                temp = np.copy(iR_pix[index][0])
                temp[nan_index] = np.nan
                iR_pix_plot = np.vstack((iR_pix_plot,temp))

                temp = np.copy(fR_pix[index][0])
                temp[nan_index] = np.nan
                fR_pix_plot = np.vstack((fR_pix_plot,temp))

                strain_matrix_plot = np.copy(strain_matrix[index][0])
                temp = np.copy(strain_matrix_plot[:,0,0])
                temp[nan_index] = np.nan
                e11c = np.vstack((e11c,temp))
                temp = np.copy(strain_matrix_plot[:,1,1])
                temp[nan_index] = np.nan
                e22c = np.vstack((e22c,temp))
                temp = np.copy(strain_matrix_plot[:,2,2])
                temp[nan_index] = np.nan
                e33c = np.vstack((e33c,temp))
                temp = np.copy(strain_matrix_plot[:,0,1])
                temp[nan_index] = np.nan
                e12c = np.vstack((e12c,temp))
                temp = np.copy(strain_matrix_plot[:,0,2])
                temp[nan_index] = np.nan
                e13c = np.vstack((e13c,temp))
                temp = np.copy(strain_matrix_plot[:,1,2])
                temp[nan_index] = np.nan
                e23c = np.vstack((e23c,temp))
                ##
                strain_matrixs_plot = np.copy(strain_matrixs[index][0])
                temp = np.copy(strain_matrixs_plot[:,0,0])
                temp[nan_index] = np.nan
                e11s = np.vstack((e11s,temp))
                temp = np.copy(strain_matrixs_plot[:,1,1])
                temp[nan_index] = np.nan
                e22s = np.vstack((e22s,temp))
                temp = np.copy(strain_matrixs_plot[:,2,2])
                temp[nan_index] = np.nan
                e33s = np.vstack((e33s,temp))
                temp = np.copy(strain_matrixs_plot[:,0,1])
                temp[nan_index] = np.nan
                e12s = np.vstack((e12s,temp))
                temp = np.copy(strain_matrixs_plot[:,0,2])
                temp[nan_index] = np.nan
                e13s = np.vstack((e13s,temp))
                temp = np.copy(strain_matrixs_plot[:,1,2])
                temp[nan_index] = np.nan
                e23s = np.vstack((e23s,temp))

        spots_len_plot = spots_len_plot.flatten()
        mr_plot = mr_plot.flatten()
        iR_pix_plot = iR_pix_plot.flatten()
        fR_pix_plot = fR_pix_plot.flatten() 
        e11c = e11c.flatten()
        e22c = e22c.flatten()
        e33c = e33c.flatten()
        e12c = e12c.flatten()
        e13c = e13c.flatten()
        e23c = e23c.flatten()
        e11s = e11s.flatten()
        e22s = e22s.flatten()
        e33s = e33s.flatten()
        e12s = e12s.flatten()
        e13s = e13s.flatten()
        e23s = e23s.flatten()
        
        try:
            if plot_ == "crystal_frame":
                title = "strain Crystal reference "+"("+material[matid]+")"
                fig = plt.figure() 
                fig.suptitle(title)
                axs = fig.subplots(2, 3)
                axs[0, 0].set_title(r"$\epsilon_{11}$", loc='center', fontsize=8)
                axs[0, 0].hist(e11c, bins=bins)
                axs[0, 0].set_ylabel('Frequency', fontsize=8)
                axs[0, 0].tick_params(axis='both', which='major', labelsize=8)
                axs[0, 0].tick_params(axis='both', which='minor', labelsize=8)

                axs[0, 1].set_title(r"$\epsilon_{22}$", loc='center', fontsize=8)
                axs[0, 1].hist(e22c, bins=bins)
                axs[0, 1].set_ylabel('Frequency', fontsize=8)
                axs[0, 1].tick_params(axis='both', which='major', labelsize=8)
                axs[0, 1].tick_params(axis='both', which='minor', labelsize=8)

                axs[0, 2].set_title(r"$\epsilon_{33}$", loc='center', fontsize=8)
                axs[0, 2].hist(e33c, bins=bins)
                axs[0, 2].set_ylabel('Frequency', fontsize=8)
                axs[0, 2].tick_params(axis='both', which='major', labelsize=8)
                axs[0, 2].tick_params(axis='both', which='minor', labelsize=8)

                axs[1, 0].set_title(r"$\epsilon_{12}$", loc='center', fontsize=8)
                axs[1, 0].hist(e12c, bins=bins)
                axs[1, 0].set_ylabel('Frequency', fontsize=8)
                axs[1, 0].tick_params(axis='both', which='major', labelsize=8)
                axs[1, 0].tick_params(axis='both', which='minor', labelsize=8)

                axs[1, 1].set_title(r"$\epsilon_{13}$", loc='center', fontsize=8)
                axs[1, 1].hist(e13c, bins=bins)
                axs[1, 1].set_ylabel('Frequency', fontsize=8)
                axs[1, 1].tick_params(axis='both', which='major', labelsize=8)
                axs[1, 1].tick_params(axis='both', which='minor', labelsize=8)

                axs[1, 2].set_title(r"$\epsilon_{23}$", loc='center', fontsize=8)
                axs[1, 2].hist(e23c, bins=bins)
                axs[1, 2].set_ylabel('Frequency', fontsize=8)
                axs[1, 2].tick_params(axis='both', which='major', labelsize=8)
                axs[1, 2].tick_params(axis='both', which='minor', labelsize=8)
                plt.tight_layout()
                plt.show()
                #plt.savefig(title+"_"+material_id[matid]+'.png', format='png', dpi=1000) 
                #plt.close(fig)
        except:
            continue
        try:
            if plot_ == "sample_frame":
                title = "strain Sample reference "+"("+material[matid]+")"
                fig = plt.figure()
                fig.suptitle(title)
                axs = fig.subplots(2, 3)
                axs[0, 0].set_title(r"$\epsilon_{11}$", loc='center', fontsize=8)
                axs[0, 0].hist(e11s, bins=bins)
                axs[0, 0].set_ylabel('Frequency', fontsize=8)
                axs[0, 0].tick_params(axis='both', which='major', labelsize=8)
                axs[0, 0].tick_params(axis='both', which='minor', labelsize=8)
                axs[0, 0].set_xlim(-0.5,0.5)

                axs[0, 1].set_title(r"$\epsilon_{22}$", loc='center', fontsize=8)
                axs[0, 1].hist(e22s, bins=bins)
                axs[0, 1].set_ylabel('Frequency', fontsize=8)
                axs[0, 1].tick_params(axis='both', which='major', labelsize=8)
                axs[0, 1].tick_params(axis='both', which='minor', labelsize=8)
                axs[0, 1].set_xlim(-0.5,0.5)

                axs[0, 2].set_title(r"$\epsilon_{33}$", loc='center', fontsize=8)
                axs[0, 2].hist(e33s, bins=bins)
                axs[0, 2].set_ylabel('Frequency', fontsize=8)
                axs[0, 2].tick_params(axis='both', which='major', labelsize=8)
                axs[0, 2].tick_params(axis='both', which='minor', labelsize=8)
                axs[0, 2].set_xlim(-0.5,0.5)

                axs[1, 0].set_title(r"$\epsilon_{12}$", loc='center', fontsize=8)
                axs[1, 0].hist(e12s, bins=bins)
                axs[1, 0].set_ylabel('Frequency', fontsize=8)
                axs[1, 0].tick_params(axis='both', which='major', labelsize=8)
                axs[1, 0].tick_params(axis='both', which='minor', labelsize=8)
                axs[1, 0].set_xlim(-0.5,0.5)

                axs[1, 1].set_title(r"$\epsilon_{13}$", loc='center', fontsize=8)
                axs[1, 1].hist(e13s, bins=bins)
                axs[1, 1].set_ylabel('Frequency', fontsize=8)
                axs[1, 1].tick_params(axis='both', which='major', labelsize=8)
                axs[1, 1].tick_params(axis='both', which='minor', labelsize=8)
                axs[1, 1].set_xlim(-0.5,0.5)

                axs[1, 2].set_title(r"$\epsilon_{23}$", loc='center', fontsize=8)
                axs[1, 2].hist(e23s, bins=bins)
                axs[1, 2].set_ylabel('Frequency', fontsize=8)
                axs[1, 2].tick_params(axis='both', which='major', labelsize=8)
                axs[1, 2].tick_params(axis='both', which='minor', labelsize=8)
                axs[1, 2].set_xlim(-0.5,0.5)
                plt.tight_layout()
                plt.show()
                #plt.savefig(title+"_"+material_id[matid]+'.png', format='png', dpi=1000) 
                #plt.close(fig)
        except:
            continue
            
plot_=['select a reference frame', 'crystal_frame', 'sample_frame']    
    
style = {'description_width': 'initial', 'widget_width': 'initial'}
match_tol_mat0=widgets.FloatSlider(min=0,max=100,step=0.1,value=0, continuous_update=False, style=style, description='Matching rate threshold '+material_)
pixel_residues_mat0=widgets.FloatSlider(min=0,max=np.max(fR_pix),step=0.01,value=np.max(fR_pix), continuous_update=False, style=style, description='Pixel residues threshold '+material_)
NbofSpots_mat0=widgets.IntSlider(min=0,max=np.max(spots_len),step=1,value=0, continuous_update=False, style=style, description='Spots threshold '+material_)
bins_mat0=widgets.IntSlider(min=1,max=100,step=1,value=20, continuous_update=False, style=style, description='Bins '+material_)
match_tol_mat1=widgets.FloatSlider(min=0,max=100,step=0.1,value=0, continuous_update=False, style=style, description='Matching rate threshold '+material1_)
pixel_residues_mat1=widgets.FloatSlider(min=0,max=np.max(fR_pix),step=0.01,value=np.max(fR_pix), continuous_update=False, style=style, description='Pixel residues threshold '+material1_)
NbofSpots_mat1=widgets.IntSlider(min=0,max=np.max(spots_len),step=1,value=0, continuous_update=False, style=style, description='Spots threshold '+material1_)
bins_mat1=widgets.IntSlider(min=1,max=100,step=1,value=20, continuous_update=False, style=style, description='Bins '+material1_)

ip = interactive(f1, plot_=plot_, match_tol0=match_tol_mat0, pixel_residues0=pixel_residues_mat0, NbofSpots0=NbofSpots_mat0, bins0=bins_mat0, \
                match_tol1=match_tol_mat1, pixel_residues1=pixel_residues_mat1, NbofSpots1=NbofSpots_mat1, bins1=bins_mat1)


left_box = widgets.VBox(ip.children[1:5])
right_box = widgets.VBox(ip.children[5:9])
display(widgets.HBox([left_box, right_box]))#Show all controls
display(ip.children[0])
display(ip.children[-1])#Show the output

##TODO add a save button to save images at the end

Dropdown(description='plot_', options=('select a reference frame', 'crystal_frame', 'sample_frame'), value='se…

Output()

## 2D plots of strain components
### Idea is similar, you can now put orientation data or any other data and plot them as 2D map

In [3]:
## Lets plot the IPF maps
def myfmt(x, pos):
    return '{0:.3f}%'.format(x)

def f(plots_ = "z", match_tol=1, pixel_residues=1, NbofSpots=20):
    fig, ax = plt.subplots(ub_matricies, len(np.unique(mat_global))-1, figsize=(10,10))
    
    for index in range(ub_matricies):
        for ii in range(len(np.unique(mat_global[index][0]))):
            mat_index1 = mat_global[index][0]
            mask_ = np.where(mat_index1 != ii+1)[0]
            nan_index = np.where(match_rate[index][0] < match_tol)[0]
            nan_index1 = np.where(fR_pix[index][0].flatten() > pixel_residues)[0]
            nan_index2 = np.where(spots_len[index][0] < NbofSpots)[0]

            if plots_ == "ipfZ":
                col_plot = np.copy(col[index][0])
                title = "Grain "+str(index)+" IPF Z map" +" ("+material[ii]+")"

            elif plots_ == "ipfX":
                col_plot = np.copy(colx[index][0])
                title = "Grain "+str(index)+" IPF X map" +" ("+material[ii]+")"

            elif plots_ == "ipfY":
                col_plot = np.copy(coly[index][0])
                title = "Grain "+str(index)+" IPF Y map" +" ("+material[ii]+")"
            
            elif plots_ == "Ceps11":
                col_plot = np.copy(strain_matrix[index][0][:,0,0])
                title = "Grain "+str(index)+" Crystal eps 11" +" ("+material[ii]+")"
            elif plots_ == "Ceps12":
                col_plot = np.copy(strain_matrix[index][0][:,0,1])
                title = "Grain "+str(index)+" Crystal eps 12" +" ("+material[ii]+")"
            elif plots_ == "Ceps13":
                col_plot = np.copy(strain_matrix[index][0][:,0,2])
                title = "Grain "+str(index)+" Crystal eps 13" +" ("+material[ii]+")"
            elif plots_ == "Ceps21":
                col_plot = np.copy(strain_matrix[index][0][:,1,0])
                title = "Grain "+str(index)+" Crystal eps 21" +" ("+material[ii]+")"
            elif plots_ == "Ceps22":
                col_plot = np.copy(strain_matrix[index][0][:,1,1])
                title = "Grain "+str(index)+" Crystal eps 22" +" ("+material[ii]+")"
            elif plots_ == "Ceps23":
                col_plot = np.copy(strain_matrix[index][0][:,1,2])
                title = "Grain "+str(index)+" Crystal eps 23" +" ("+material[ii]+")"
            elif plots_ == "Ceps31":
                col_plot = np.copy(strain_matrix[index][0][:,2,0])
                title = "Grain "+str(index)+" Crystal eps 31" +" ("+material[ii]+")"
            elif plots_ == "Ceps32":
                col_plot = np.copy(strain_matrix[index][0][:,2,1])
                title = "Grain "+str(index)+" Crystal eps 32" +" ("+material[ii]+")"
            elif plots_ == "Ceps33":
                col_plot = np.copy(strain_matrix[index][0][:,2,2])
                title = "Grain "+str(index)+" Crystal eps 33" +" ("+material[ii]+")"
            
            elif plots_ == "Seps11":
                col_plot = np.copy(strain_matrixs[index][0][:,0,0])
                title = "Grain "+str(index)+" Sample eps 11" +" ("+material[ii]+")"
            elif plots_ == "Seps12":
                col_plot = np.copy(strain_matrixs[index][0][:,0,1])
                title = "Grain "+str(index)+" Sample eps 12" +" ("+material[ii]+")"
            elif plots_ == "Seps13":
                col_plot = np.copy(strain_matrixs[index][0][:,0,2])
                title = "Grain "+str(index)+" Sample eps 13" +" ("+material[ii]+")"
            elif plots_ == "Seps21":
                col_plot = np.copy(strain_matrixs[index][0][:,1,0])
                title = "Grain "+str(index)+" Sample eps 21" +" ("+material[ii]+")"
            elif plots_ == "Seps22":
                col_plot = np.copy(strain_matrixs[index][0][:,1,1])
                title = "Grain "+str(index)+" Sample eps 22" +" ("+material[ii]+")"
            elif plots_ == "Seps23":
                col_plot = np.copy(strain_matrixs[index][0][:,1,2])
                title = "Grain "+str(index)+" Sample eps 23" +" ("+material[ii]+")"
            elif plots_ == "Seps31":
                col_plot = np.copy(strain_matrixs[index][0][:,2,0])
                title = "Grain "+str(index)+" Sample eps 31" +" ("+material[ii]+")"
            elif plots_ == "Seps32":
                col_plot = np.copy(strain_matrixs[index][0][:,2,1])
                title = "Grain "+str(index)+" Sample eps 32" +" ("+material[ii]+")"
            elif plots_ == "Seps33":
                col_plot = np.copy(strain_matrixs[index][0][:,2,2])
                title = "Grain "+str(index)+" Sample eps 33" +" ("+material[ii]+")"
                
            if plots_ in ["ipfX", "ipfY", "ipfZ"]:
                col_plot[mask_,:] = 0,0,0
                col_plot[nan_index,:] = 0,0,0
                col_plot[nan_index1,:] = 0,0,0
                col_plot[nan_index2,:] = 0,0,0
                col_plot = col_plot.reshape((lim_x, lim_y,3))
            else:
                col_plot[mask_] = np.nan
                col_plot[nan_index] = np.nan
                col_plot[nan_index1] = np.nan
                col_plot[nan_index2] = np.nan
                col_plot = col_plot.reshape((lim_x, lim_y))

            if ub_matricies == 1 and len(np.unique(mat_global)) == 1:
                ax.set_title(title, loc='center', fontsize=20)
                im = ax.imshow(col_plot, origin='lower')
                ax.set_xticks([])
                ax.set_yticks([])
                if plots_ not in ["ipfX", "ipfY", "ipfZ"]:
                    divider = make_axes_locatable(ax)
                    cax = divider.append_axes('right', size='5%', pad=0.05)
                    cbar = fig.colorbar(im, cax=cax, orientation='vertical', format=ticker.FuncFormatter(myfmt))
                    cbar.ax.tick_params(labelsize=20) 
            else:
                ax[index, ii].set_title(title, loc='center', fontsize=8)
                im = ax[index, ii].imshow(col_plot, origin='lower')
                ax[index, ii].set_xticks([])
                ax[index, ii].set_yticks([])
                #for axs in ax.flat:
                #    axs.label_outer()
                if plots_ not in ["ipfX", "ipfY", "ipfZ"]:
                    divider = make_axes_locatable(ax[index, ii])
                    cax = divider.append_axes('right', size='5%', pad=0.05)
                    cbar = fig.colorbar(im, cax=cax, orientation='vertical', format=ticker.FuncFormatter(myfmt))
                    cbar.ax.tick_params(labelsize=20) 
    #plt.grid()
    fig.tight_layout()
    plt.show()

plots_=['Ceps11', 'Ceps12', 'Ceps13', 'Ceps21', 'Ceps22', 'Ceps23', 'Ceps31', 'Ceps32', 'Ceps33',
       'Seps11', 'Seps12', 'Seps13', 'Seps21', 'Seps22', 'Seps23', 'Seps31', 'Seps32', 'Seps33',
       'ipfX', 'ipfY', 'ipfZ']    
    
style = {'description_width': 'initial', 'widget_width': 'initial'}
match_tol=widgets.FloatSlider(min=0,max=100,step=0.1,value=1, continuous_update=True, style=style) ##step of 1pixel
pixel_residues=widgets.FloatSlider(min=0,max=np.max(fR_pix),step=0.01,value=np.max(fR_pix)//2, continuous_update=True, style=style) ##step of 1pixel
NbofSpots = widgets.IntSlider(min=0,max=np.max(spots_len),step=1,value=np.max(spots_len)//2, continuous_update=True, style=style) ##step of 1pixel

ip = interactive(f, plots_ = plots_, match_tol=match_tol, pixel_residues=pixel_residues, NbofSpots=NbofSpots)
display(widgets.VBox(ip.children[:-1]))

display(ip.children[-1])

## for colorbar ipyscales
## TODO cleaner plots

Output()